In [3]:
import scipy
import numpy as np
import csv
import cv2
import os
from tqdm import tqdm_notebook as tqdm
import pickle


from scoring.score import score, convert_to_rectangle_list
from scoring.matching import Matching
from scoring.rectangle import Rectangle
from utils.utils import plot_one_box

In [4]:
# Get all ground truth bboxes
mat = scipy.io.loadmat('scoring/ground_truth.mat')
gt_coords, gt_chips, gt_classes = mat['gt_coords'], mat['gt_chips'], mat['gt_classes']
gt_unique = np.unique(gt_classes.astype(np.int64))

In [14]:
# Get class number to name mappings
class_nums = [ 11.0, 12.0, 13.0, 15.0, 17.0, 18.0, 19.0, 20.0, 21.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 40.0, 41.0, 42.0, 44.0, 45.0, 47.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 71.0, 72.0, 73.0, 74.0, 76.0, 77.0, 79.0, 83.0, 84.0, 86.0, 89.0, 91.0, 93.0, 94.0]
with open('data/xview.names') as f:
    lines = f.readlines()

class_num_to_name = { int(c): lines[i].strip() for i, c in enumerate(class_nums) }
print(class_num_to_name)

{11: 'Fixed-wing Aircraft', 12: 'Small Aircraft', 13: 'Cargo Plane', 15: 'Helicopter', 17: 'Passenger Vehicle', 18: 'Small Car', 19: 'Bus', 20: 'Pickup Truck', 21: 'Utility Truck', 23: 'Truck', 24: 'Cargo Truck', 25: 'Truck w/Box', 26: 'Truck Tractor', 27: 'Trailer', 28: 'Truck w/Flatbed', 29: 'Truck w/Liquid', 32: 'Crane Truck', 33: 'Railway Vehicle', 34: 'Passenger Car', 35: 'Cargo Car', 36: 'Flat Car', 37: 'Tank car', 38: 'Locomotive', 40: 'Maritime Vessel', 41: 'Motorboat', 42: 'Sailboat', 44: 'Tugboat', 45: 'Barge', 47: 'Fishing Vessel', 49: 'Ferry', 50: 'Yacht', 51: 'Container Ship', 52: 'Oil Tanker', 53: 'Engineering Vehicle', 54: 'Tower crane', 55: 'Container Crane', 56: 'Reach Stacker', 57: 'Straddle Carrier', 59: 'Mobile Crane', 60: 'Dump Truck', 61: 'Haul Truck', 62: 'Scraper/Tractor', 63: 'Front loader/Bulldozer', 64: 'Excavator', 65: 'Cement Mixer', 66: 'Ground Grader', 71: 'Hut/Tent', 72: 'Shed', 73: 'Building', 74: 'Aircraft Hangar', 76: 'Damaged Building', 77: 'Facility

### Get stats from an image
Adapted from scoring/score.py, `score` function.

In [40]:
suffix = ''#'_ed_area_lcls_n1024_b25_x100_continuous'
output_folder = 'output' + suffix

In [41]:
def get_detections(img_filename):
    path = './' + output_folder + '/' + img_filename + '.txt'
    
    with open(path, 'r') as f:
        arr = np.array(list(csv.reader(f, delimiter=" ")))
        assert arr.shape[0] != 0 # make sure file isn't empty
        
        arr = arr[:, :6].astype(np.float64)
        threshold = 0
        arr = arr[arr[:, 5] > threshold] # Claire comment: confidence threshold (currently 0)
        det_classes = list(arr[:, 4])
        num_preds = arr.shape[0]

        if np.any(arr[:, :4] < 0):
            raise ValueError('Bounding boxes cannot be negative.')

        if np.any(arr[:, 5] < 0) or np.any(arr[:, 5] > 1):
            raise ValueError('Confidence scores should be between 0 and 1.')
            
        det_box = arr[:, :4]
        det_scores = arr[:, 5]
        det_cls = arr[:, 4]
    
    return det_box, det_scores, det_cls

In [10]:
def evaluate_one(img_filename, iou_threshold=.5):
    assert (iou_threshold < 1 and iou_threshold > 0)
    
    # Get predicted bboxes
    det_box, det_scores, det_cls = get_detections(img_filename)
        
    # Get ground truth bboxes
    gt_box = gt_coords[(gt_chips == img_filename).flatten()]
    gt_cls = gt_classes[(gt_chips == img_filename)]
    
    print('%d detected boxes, %d ground truth boxes' % (len(det_box), len(gt_box)))
    
    per_class_data = {}
    for i in gt_unique:
        per_class_data[i] = {
            'scores': [],
            'rects': [],
            'gt_rects': [],
            'rects_matched': [],
            'gt_rects_matched': [],
        }
    
    # For each class, match predicted and GT bboxes
    for i in gt_unique:
        s = det_scores[det_cls == i] # scores for this class
        ssort = np.argsort(s)[::-1] # sorted score indices
        sorted_scores = s[ssort].tolist() # sorted scores

        # bounding boxes for this class (gt and predicted)
        gt_box_i_cls = gt_box[gt_cls == i].flatten().tolist()
        det_box_i_cls = det_box[det_cls == i]
        det_box_i_cls = det_box_i_cls[ssort].flatten().tolist()

        # convert into Rectangle objects (has helper functions)
        gt_rects, _, _ = convert_to_rectangle_list(gt_box_i_cls)
        rects, _, small_indices = convert_to_rectangle_list(det_box_i_cls)

        # match all Rectangles
        matching = Matching(gt_rects, rects)
        rects_matched, gt_matched = matching.greedy_match(iou_threshold) # Claire comment: returns two boolean lists
              
        tp_indices = [i for i in range(len(rects)) if rects_matched[i] == True] if len(rects_matched) > 0 else []
        fp_indices = [i for i in range(len(rects)) if rects_matched[i] == False] if len(rects_matched) > 0 else []
        fn_indices = [i for i in range(len(gt_rects)) if gt_matched[i] == False] if len(gt_matched) > 0 else []
        
        # Rectangles by match type
        tp_rects = [rects[i] for i in tp_indices]
        fp_rects = [rects[i] for i in fp_indices]
        fn_rects = [gt_rects[i] for i in fn_indices]
        
        # Scores by match type
        tp_scores = [sorted_scores[i] for i in tp_indices]
        fp_scores = [sorted_scores[i] for i in fp_indices]
                
            
        per_class_data[i] = {
            'scores': sorted_scores,
            'rects': rects,
            'gt_rects': gt_rects,
            'rects_matched': rects_matched,
            'gt_rects_matched': gt_matched,
            # Counts by match type
            'num_tp': len(tp_rects),
            'num_fp': len(fp_rects),
            'num_fn': len(fn_rects),
            # Rects by match type
            'tp_rects': tp_rects,
            'fp_rects': fp_rects,
            'fn_rects': fn_rects,
            # Scores by match type
            'tp_scores': tp_scores,
            'fp_scores': fp_scores,
        }

    return per_class_data


In [33]:
per_class_data = evaluate_one('886.tif')

4029 detected boxes, 1985 ground truth boxes


In [9]:
print(per_class_data[17])

{'scores': [0.558536, 0.467016, 0.400027, 0.390104, 0.388396, 0.383982, 0.381459, 0.376618, 0.366816, 0.366602, 0.358498, 0.349187, 0.344561, 0.339413, 0.338652, 0.332373, 0.327078, 0.324892, 0.322354, 0.319129, 0.318486, 0.308683], 'rects': [<scoring.rectangle.Rectangle object at 0x7f7ffcbdd550>, <scoring.rectangle.Rectangle object at 0x7f7ffcbdd240>, <scoring.rectangle.Rectangle object at 0x7f7ffcbdd5f8>, <scoring.rectangle.Rectangle object at 0x7f7ffcbdd358>, <scoring.rectangle.Rectangle object at 0x7f7ffcbdd588>, <scoring.rectangle.Rectangle object at 0x7f7ffcbdd748>, <scoring.rectangle.Rectangle object at 0x7f7ffcbdd438>, <scoring.rectangle.Rectangle object at 0x7f7ffcbdd320>, <scoring.rectangle.Rectangle object at 0x7f7ffcbdd390>, <scoring.rectangle.Rectangle object at 0x7f7ffcbdd400>, <scoring.rectangle.Rectangle object at 0x7f7ffcbdd6a0>, <scoring.rectangle.Rectangle object at 0x7f7ffcbdd470>, <scoring.rectangle.Rectangle object at 0x7f7ffcbdd0b8>, <scoring.rectangle.Rectangle 

### Draw true positives, false positives, false negatives on img

In [42]:
def plot_bboxes(img_filename, per_class_data):
    img_path = '../../xview_data/one_test_image/' + img_filename
    img = cv2.imread(img_path)
    
    output_path = 'bboxes/' + img_filename
    
    # Draw bounding boxes and labels of detections
    for c in per_class_data:
        class_name = class_num_to_name[c] if c in class_num_to_name else 'Unknown'
        data = per_class_data[c]
#         print('%s' % (class_name))
        
        # True positives
        color = [0, 255, 0]
        tp_scores = data['tp_scores']
#         print('\tTrue positives: %d' % (data['num_tp']))
        for i, rect in enumerate(data['tp_rects']):
            xmin, ymin, xmax, ymax = rect.coords
            x1, y1, x2, y2 = max(xmin, 0), max(ymin, 0), max(xmax, 0), max(ymax, 0)
            label = '%s %.2f' % (class_name, tp_scores[i])
            plot_one_box([x1, y1, x2, y2], img, label=label, color=color, line_thickness=1)
        
#         False positives
        color = [0, 0, 255]
        fp_scores = data['fp_scores']
#         print('\tFalse positives: %d' % (data['num_fp']))
        for i, rect in enumerate(data['fp_rects']):
            xmin, ymin, xmax, ymax = rect.coords
            x1, y1, x2, y2 = max(xmin, 0), max(ymin, 0), max(xmax, 0), max(ymax, 0)
            label = '%s %.2f' % (class_name, fp_scores[i])
            plot_one_box([x1, y1, x2, y2], img, label=label, color=color, line_thickness=1)
    
        # False negatives
        color = [255, 0, 0]
#         print('\tFalse negatives: %d' % (data['num_fn']))
        for rect in data['fn_rects']:
            xmin, ymin, xmax, ymax = rect.coords
            x1, y1, x2, y2 = max(xmin, 0), max(ymin, 0), max(xmax, 0), max(ymax, 0)
            plot_one_box([x1, y1, x2, y2], img, label=class_name, color=color, line_thickness=1)
        
    cv2.imwrite(output_path.replace('.tif', '.jpg'), img)

In [39]:
plot_bboxes('886.tif', per_class_data)

### Run for all images

In [43]:
test_imgs = os.listdir('../../xview_data/test_images')

all_data = {}
for file in tqdm(test_imgs):
    if not file.endswith('.tif'):
        continue
        
    if not os.path.exists('./' + output_folder + '/' + file + '.txt'):
        continue
        
    per_class_data = evaluate_one(file)
    plot_bboxes(file, per_class_data)
    all_data[file] = per_class_data
    
pickle.dump(all_data, open('test_evaluation' + suffix + '.pkl', 'wb'))

168 detected boxes, 36 ground truth boxes
1002 detected boxes, 522 ground truth boxes
78 detected boxes, 7 ground truth boxes
46 detected boxes, 19 ground truth boxes
69 detected boxes, 138 ground truth boxes
48 detected boxes, 43 ground truth boxes
1892 detected boxes, 795 ground truth boxes
4 detected boxes, 1 ground truth boxes
23 detected boxes, 10 ground truth boxes
25 detected boxes, 14 ground truth boxes
1003 detected boxes, 351 ground truth boxes
38642 detected boxes, 827 ground truth boxes
369 detected boxes, 279 ground truth boxes
241 detected boxes, 44 ground truth boxes
75 detected boxes, 1 ground truth boxes
4 detected boxes, 2 ground truth boxes
20 detected boxes, 7 ground truth boxes
845 detected boxes, 488 ground truth boxes
49 detected boxes, 14 ground truth boxes
488 detected boxes, 302 ground truth boxes
262 detected boxes, 172 ground truth boxes
5234 detected boxes, 5346 ground truth boxes
392 detected boxes, 227 ground truth boxes
5159 detected boxes, 2855 ground t

In [44]:
test_data = pickle.load(open('test_evaluation' + suffix + '.pkl', 'rb'))

In [45]:
# Write to csv
with open('test_evaluation' + suffix + '.csv', mode='w') as file:
    file_writer = csv.writer(file, delimiter=',')
    
    for img in tqdm(test_data):
        for c in test_data[img]:
            class_name = class_num_to_name[c] if c in class_num_to_name else 'Unknown'
            data = test_data[img][c]
            
            tp_scores = data['tp_scores']
            for i, rect in enumerate(data['tp_rects']):
                file_writer.writerow([
                    img,
                    class_name,
                    rect.area(),
                    'tp',
                    tp_scores[i]
                ])
                
            fp_scores = data['fp_scores']
            for i, rect in enumerate(data['fp_rects']):
                file_writer.writerow([
                    img,
                    class_name,
                    rect.area(),
                    'fp',
                    fp_scores[i]
                ])
                
            for i, rect in enumerate(data['fn_rects']):
                file_writer.writerow([
                    img,
                    class_name,
                    rect.area(),
                    'fn',
                    0
                ])
        

In [33]:
total_num_gt = 0

for c in test_data['1799.tif']:
    data = test_data['1799.tif'][c]
#     print('%d: %d == %d + %d' % (c, len(data['gt_rects']), data['num_tp'], data['num_fn']))
    assert(len(data['gt_rects']) == data['num_tp'] + data['num_fn'])
    total_num_gt += len(data['gt_rects'])
    
print(total_num_gt)

43


In [22]:
from models import Darknet
import torch

cuda = True
device = torch.device('cuda:0' if cuda else 'cpu')

model = Darknet('cfg/c60_a30symmetric.cfg', 800)
checkpoint = torch.load('weights/weights_30_original.pt', map_location='cpu')
model.load_state_dict(checkpoint['model'])
model.to(device).eval()
del checkpoint

In [26]:
from utils.utils import build_targets

def evaluate_one_utils(img_filename):
    # Get predicted bboxes
    det_box, det_scores, det_cls = get_detections(img_filename)
    # Get ground truth bboxes
    gt_box = gt_coords[(gt_chips == img_filename).flatten()]
    gt_cls = gt_classes[(gt_chips == img_filename)]
    
    nA = 3
    nC = 60
    anchor_wh = model.scaled_anchors
    nGT, nCorrect, tx, ty, tw, th, tconf, tcls = build_targets(det_box, det_conf, det_cls, target, anchor_wh, nA, nC, nG, requestPrecision)

In [27]:
evaluate_one_utils('1184.tif')

AttributeError: 'Darknet' object has no attribute 'scaled_anchors'

In [6]:
import csv
with open('bboxes.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',')
    for i in range(len(gt_coords)):
        x1, y1, x2, y2 = gt_coords[i]
        width = x2 - x1
        height = y2 - y1
        writer.writerow([width, height])